In [18]:
%pip install chromadb
%pip install llama-index chromadb
%pip install llama-index-vector-stores-chroma

Note: you may need to restart the kernel to use updated packages.
  Using cached llama_index_core-0.11.23-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.11.23-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.12.0
    Uninstalling llama-index-core-0.12.0:
      Successfully uninstalled llama-index-core-0.12.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-huggingface 0.4.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-vector-stores-chroma 0.4.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.


  Using cached llama_index_core-0.12.0-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.12.0-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.11.23
    Uninstalling llama-index-core-0.11.23:
      Successfully uninstalled llama-index-core-0.11.23
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.11.23 requires llama-index-core<0.12.0,>=0.11.23, but you have llama-index-core 0.12.0 which is incompatible.
llama-index-agent-openai 0.3.4 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.0 which is incompatible.
llama-index-cli 0.3.1 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.0 which is incompatible.
llama-index-embeddings-ollama 0.3.1 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.0 which is incompatible.
llama-index-embeddings-openai 0.2.5 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.0 which is incompatible.
llama-index-indices-managed-llama-cloud 0.4.0 requires llama-index-core<0.12.0,>=0.11.13.post1, but you have llama-index-core 0.12.0 which is incompatible.
llama-in

In [19]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.ollama import OllamaEmbedding




In [20]:
documents = SimpleDirectoryReader(input_files=['../data_uber/uber_2021.pdf']).load_data()

In [21]:
len(documents)

307

In [22]:
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",  # Replace with your desired model
    base_url="http://localhost:11434",  # Ensure Ollama is running at this endpoint
    ollama_additional_kwargs={"mirostat": 0} #Mirostat is a technique for controlling perplexity and balancing the text generation process in large language models (LLMs).
)    

In [23]:
index = VectorStoreIndex.from_documents(documents, embed_model=ollama_embedding)


In [24]:
from llama_index.core import Settings
from llama_index.llms.ollama import  Ollama

Settings.llm = Ollama(model='llama3.2:latest', base_url='http://localhost:11434',temperature=0.1)

In [25]:
from llama_index.core import get_response_synthesizer

In [26]:
# Refine Mode
response_synthesizer = get_response_synthesizer(response_mode="refine")
response = index.as_query_engine(response_synthesizer=response_synthesizer).query("What are the key highlights of Uber's 2021 report?")
print(response)

Revenue decreased by $127 million, or 94%, in the year ended December 31, 2021 compared to the same period in 2020. Adjusted EBITDA loss improved by $450 million, or 98%. These decreases were primarily due to the favorable impact of the sale of the ATG Business and the JUMP Divestiture.


In [27]:
# Compact Mode
response_synthesizer = get_response_synthesizer(response_mode="compact")
# response = index.as_query_engine(response_synthesizer=response_synthesizer).query("What are the key highlights of Uber's 2021 report?")
response = index.as_query_engine(response_synthesizer=response_synthesizer).query("Summarize Uber's 2021 performance.")
print(response)


Uber's 2021 performance was marked by significant declines in revenue and adjusted EBITDA loss. The company reported a 94% decrease in All Other revenue and an 98% improvement in All Other adjusted EBITDA loss compared to the same period in 2020. This decline was largely attributed to the favorable impact of the sale of its ATG Business in the first quarter of 2021 and the JUMP Divestiture in the second quarter of 2020.


In [28]:
# Tree Summarize Mode
response_synthesizer = get_response_synthesizer(response_mode="tree_summarize")
# response = index.as_query_engine(response_synthesizer=response_synthesizer).query("What are the key highlights of Uber's 2021 report?")
response = index.as_query_engine(response_synthesizer=response_synthesizer).query("Provide a structured summary of Uber's 2021 report.")
print(response)

**Summary of Uber's 2021 Report**

**Financial Highlights**

* Revenue: $17,455 million (up 55.6% from 2020)
* Net Loss: ($496) million (improved from a net loss of ($8,512) million in 2020)

**Segmented Financial Performance**

* Cost of Revenue: $9,351 million (up 79.4% from 2020)
* Operations and Support: $1,877 million (down 3.7% from 2020)
* Sales and Marketing: $4,789 million (up 33.5% from 2020)
* Research and Development: $2,054 million (down 6.8% from 2020)

**Key Expenses**

* Depreciation and Amortization: $902 million (up 57.1% from 2020)
* Interest Expense: ($483) million (up 5.7% from 2020)

**Shareholder Information**

* Net Loss per Share (Basic): ($0.26) per share
* Weighted-Average Shares Used to Compute Net Loss per Share: 1,892,546 shares

**Governance and Compensation**

* The company's governance structure and executive compensation practices are disclosed in the accompanying proxy statement.

**Exhibits and Financial Statements**

* The annual report includes con

In [29]:
# Accumulate Mode
response_synthesizer = get_response_synthesizer(response_mode="accumulate")
response = index.as_query_engine(response_synthesizer=response_synthesizer).query("Retrieve all data points related to Uber's 2021 report.")
print(response)


Response 1: Year Ended December 31, 2019:
- Revenue: $11,139 million
- Costs and expenses:
  - Cost of revenue: $5,154 million
  - Operations and support: $1,819 million
  - Sales and marketing: $3,583 million
  - Research and development: $2,205 million
  - General and administrative: $2,666 million
- Depreciation and amortization: $575 million
- Loss from operations: $(4,863) million

Year Ended December 31, 2020:
- Revenue: $17,455 million
- Costs and expenses:
  - Cost of revenue: $9,351 million
  - Operations and support: $1,877 million
  - Sales and marketing: $4,789 million
  - Research and development: $2,054 million
  - General and administrative: $2,316 million
- Depreciation and amortization: $902 million
- Loss from operations: $(3,834) million

Year Ended December 31, 2021:
- Revenue: $17,455 million
- Costs and expenses:
  - Cost of revenue: $9,351 million
  - Operations and support: $1,877 million
  - Sales and marketing: $4,789 million
  - Research and development: $2,0

In [30]:
# Compact Accumulate Mode
response_synthesizer = get_response_synthesizer(response_mode="compact_accumulate")
response = index.as_query_engine(response_synthesizer=response_synthesizer).query("Provide a concise version of all data points in Uber's 2021 report.")
print(response)


Response 1: Here is a concise summary of the key data points from Uber's 2021 report:

**Revenue:** $17,455 million (up 55% from 2020)

**Costs and expenses:**

* Cost of revenue: $9,351 million
* Operations and support: $1,877 million
* Sales and marketing: $4,789 million
* Research and development: $2,054 million
* General and administrative: $2,316 million

**Loss from operations:** ($3,834) million (improved from 2020)

**Interest expense:** ($483) million

**Other income (expense), net:** $3,292 million

**Key metrics:**

* Monthly Active Platform Consumers (MAPCs): Not provided
* Trips: Not provided

**Net loss and per share data:**

* Net loss: ($570) million
* Net loss per share:
	+ Basic: ($0.26)
	+ Diluted: ($0.29)

Note that some key metrics, such as MAPCs and trips, are not explicitly stated in the report.
